In [2]:
from glob import glob
import ctypes
import struct
import sys
from pathlib import Path

import numpy as np

import vtu
PyVtu=vtu.PyVtu
import small_functions
import ts_wrapper as ts

In [6]:
path_to_trisurf = Path('/opt/workspace/msc_project/cluster-trisurf')

In [3]:
vesicle = ts.parseDump('/mnt/c/Users/yoavr/Desktop/timestep_000087.vtu')

In [4]:
vesicle.contents.cm

In [8]:
tape=ts.parsetape(str(path_to_trisurf/'src/tape'))

In [32]:
fields = tape.contents._fields_
contents = [tape.contents.__getattribute__(field[0]) for field in fields]
str_field = [str(field[0]) for field in fields]
cont_str = [repr(content) for content in contents]
print('\n'.join(x+":"+y for x,y in zip(str_field,cont_str)))

tape_text:<ctypes.LP_c_char object at 0x7fc73c5c9940>
R_nucleus:0.0
R_nucleusX:0.0
R_nucleusY:0.0
R_nucleusZ:0.0
kxA0:1.0
kxV0:1.0
V0:0.0
A0:0.0
Vfraction:1.0
constvolprecision:1e-14
xk0:20.0
xk2:-20.0
dmax:1.7
dmin_interspecies:1.2
stepsize:0.15
kspring:800.0
xi:100.0
pressure:0.0
c0:0.5
d0:0.5
w:1.0
F:1.0
plane_d:10.0
plane_F:1.0
vicsek_strength:0.1
vicsek_radius:4.0
adhesion_z:-5.0
adhesion_cutoff:1.0
adhesion_strength:1.0
adhesion_radius:5.0
adhesion_scale:5.0
adhesion_factor:2.0
min_dihedral_angle_cosine:0.1
mcsweeps:1000
random_seed:0
iterations:10
inititer:0
nshell:10
ncxmax:100
ncymax:100
nczmax:100
number_of_vertices_with_c0:50
npoly:0
nmono:20
internal_poly:0
nfil:0
nfono:3
shc:0
pressure_switch:b'\x00'
volume_switch:b'\x00'
area_switch:b'\x00'
quiet:b'\x00'
plane_confinment_switch:b'\x00'
allow_center_mass_movement:b'\x00'
force_balance_along_z_axis:b'\x00'
adhesion_geometry:b'\x00'
adhesion_model:b'\x00'
bond_model:b'\x00'
curvature_model:b'\x0f'
force_model:b'\x00'


In [40]:
np.array([vesicle.contents.vlist.contents.vtx[i].contents.c for i in range(50,100)])

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.])

In [41]:
vtx=vesicle.contents.vlist.contents.vtx[97]
vtx_old=ts.ts_vertex(x=0,y=0,z=0)

In [42]:
vtx.contents.z

-0.8323908581740378

In [43]:
import ctypes

In [44]:
ctypes.pointer(vtx_old)

In [45]:
ts.ts.direct_force_energy.restype=ctypes.c_double
ts.ts.direct_force_energy(vesicle, vtx, ctypes.pointer(vtx_old))

0.0

In [46]:
int.from_bytes(vesicle.contents.tape.contents.force_model,sys.byteorder)

16

In [47]:
ts.ts.direct_force_energy.restype=ctypes.c_double
ts.ts.direct_force_energy(vesicle,ctypes.pointer(vtx),ctypes.pointer(vtx_old))


0.0

In [50]:
file = Path("/mnt/c/Users/yoavr/Desktop/timestep_000087.vtu")

In [51]:
v = PyVtu(file)

In [52]:
vesicle = ts.parseDump(str(file))

In [53]:
vertices = [
vesicle.contents.vlist.contents.vtx[i] for i in range(vesicle.contents.vlist.contents.n)]

In [54]:
vertices[2]

In [55]:
dist = [
ts.adhesion_geometry_distance(vesicle, vtx) for vtx in vertices]

In [56]:
int.from_bytes(vesicle.contents.tape.contents.adhesion_geometry,sys.byteorder)

2

In [57]:
vesicle.contents.tape.contents.adhesion_radius

10.0

In [58]:
np.array(dist)[np.array(dist)<1]

array([0.91339441, 0.96547936, 0.91553219, 0.89688824, 0.86787479])

In [59]:
dist2=(((v.pos - [[0,0,-20]])**2).sum(axis=1)**0.5-10)

In [60]:
dist=np.array(dist)

In [61]:
i=568
dist2[i],v.pos[i],dist[i],(vertices[i].contents.x,vertices[i].contents.y,vertices[i].contents.z)

(0.9133944089548862,
 array([-0.69210235, -2.19473144, -9.33199523]),
 0.9133944089548862,
 (-0.6921023463065015, -2.1947314360371157, -9.331995229129523))

In [62]:
v.type[i]

4

In [64]:
ts.adhesion_energy_diff(vesicle, vertices[i],vertices[0])

-1.25

In [65]:
ts.adhesion_geometry_side(vesicle, vertices[i])

b'\x01'

In [67]:
max_cluster_size = 450
cluster_hist = np.zeros(max_cluster_size)

In [68]:
file_regex = ("/mnt/c/Users/yoavr/Desktop/paraview_pipeline"
                "/hello_chemfarm/anisotropy/5_from_mitja/x0000_b0k0"
                "/time*")

file_regex = ("/opt/workspace/msc_project/simulations"
                "/QA_tests/throwaway"
                "/time*")

files = sorted(glob(file_regex))
files[:6]

['/opt/workspace/msc_project/simulations/QA_tests/throwaway/timestep_000000.vtu',
 '/opt/workspace/msc_project/simulations/QA_tests/throwaway/timestep_000001.vtu',
 '/opt/workspace/msc_project/simulations/QA_tests/throwaway/timestep_000002.vtu',
 '/opt/workspace/msc_project/simulations/QA_tests/throwaway/timestep_000003.vtu',
 '/opt/workspace/msc_project/simulations/QA_tests/throwaway/timestep_000004.vtu',
 '/opt/workspace/msc_project/simulations/QA_tests/throwaway/timestep_000005.vtu']

In [69]:
for file in files[:3]:
    vesicle=ts.parseDump(file)
    cstlist=ts.init_cluster_list()
    ts.clusterize_vesicle(vesicle, cstlist)    
    for j in range(cstlist.contents.n):
        size=cstlist.contents.cluster[j].contents.nvtx
        cluster_hist[size-1]+=1.0000

In [82]:
v=PyVtu(file)

In [83]:
vtx_500 = vesicle.contents.vlist.contents.vtx[500]

In [84]:
v.pos[500], vtx_500.contents.x,vtx_500.contents.y,vtx_500.contents.z

(array([ 0.05522578,  0.04100399, -6.23498369]),
 0.05522578450024107,
 0.04100399122861435,
 -6.234983690768586)

In [85]:
ts.adhesion_geometry_distance(vesicle, vesicle.contents.vlist.contents.vtx[500])

0.0

In [86]:
v=vtu.PyVtu(file)

In [87]:
for field, ty in vesicle.contents._fields_:
    if (ty in {ctypes.c_int, ctypes.c_uint, ctypes.c_double}):
        print(field,vesicle.contents.__getattribute__(field))
    elif (ty in {ctypes.c_double*3,}):
        print(field, vesicle.contents.__getattribute__(field)[:])
    else:
        print(field,vesicle.contents.__getattribute__(field))

dmax 2.8899999999999997
stepsize 0.15
cm [0.0, 0.0, 0.0]
fx 0.0
fy 0.0
fz 0.0
volume 0.0
area 0.0
spring_constant 800.0
pressure 0.0
R_nucleus 0.0
R_nucleusX 0.0
R_nucleusY 0.0
R_nucleusZ 0.0
nucleus_center [0.0, 0.0, 0.0]
tape <ts_wrapper.LP_ts_tape object at 0x7fc739e8ba40>
sphHarmonics <ts_wrapper.LP_ts_spharm object at 0x7fc739e8b7c0>
poly_list <ts_wrapper.LP_ts_poly_list object at 0x7fc739e8ba40>
filament_list <ts_wrapper.LP_ts_poly_list object at 0x7fc739e8b7c0>
vlist <ts_wrapper.LP_ts_vertex_list object at 0x7fc739e8ba40>
blist <ts_wrapper.LP_ts_bond_list object at 0x7fc739e8b7c0>
tlist <ts_wrapper.LP_ts_triangle_list object at 0x7fc739e8ba40>
clist <ts_wrapper.LP_ts_cell_list object at 0x7fc739e8b7c0>
confinement_plane <ts_wrapper.ts_confinment_plane object at 0x7fc739e8ba40>
nshell 0


In [88]:
tape_opts = small_functions.get_tape_options(v.tape)
for i, (field, ty) in enumerate(vesicle.contents.tape.contents._fields_):
    attrib = vesicle.contents.tape.contents.__getattribute__(field)
    if (ty in {ctypes.c_int, ctypes.c_uint, ctypes.c_double,ctypes.c_ulong}):
        val = attrib
    elif (ty in {ctypes.c_double*3,}):
        val = attrib[:]
    elif (ty in {ctypes.c_char,ctypes.c_byte}):
        val = int.from_bytes(attrib,sys.byteorder)
    else:
        val=attrib
    print(i,field,val,tape_opts.get(field))

0 tape_text <ctypes.LP_c_char object at 0x7fc739e8b2c0> None
1 R_nucleus 0.0 0
2 R_nucleusX 0.0 0
3 R_nucleusY 0.0 0
4 R_nucleusZ 0.0 0
5 kxA0 1.0 None
6 kxV0 1.0 None
7 V0 0.0 0
8 A0 0.0 0
9 Vfraction 1.0 None
10 constvolprecision 1e-14 1e-14
11 xk0 20.0 20
12 xk2 -20.0 -20
13 dmax 1.7 1.7
14 dmin_interspecies 1.2 1.2
15 stepsize 0.15 0.15
16 kspring 800.0 None
17 xi 100.0 100
18 pressure 0.0 0.0
19 c0 0.5 0.5
20 d0 0.5 0.5
21 w 1.0 1.0
22 F 1.0 1.0
23 plane_d 10.0 10
24 plane_F 1.0 1.0
25 vicsek_strength 0.1 0.1
26 vicsek_radius 4.0 4.0
27 adhesion_z -5.0 -5
28 adhesion_cutoff 1.0 1
29 adhesion_strength 1.0 1.0
30 adhesion_radius 5.0 5.0
31 adhesion_scale 5.0 5.0
32 adhesion_factor 2.0 2.0
33 min_dihedral_angle_cosine 0.1 0.1
34 mcsweeps 1000 1000
35 random_seed 1682609434 1682609434
36 iterations 10 10
37 inititer 0 0
38 nshell 10 10
39 ncxmax 100 None
40 ncymax 100 None
41 nczmax 100 None
42 number_of_vertices_with_c0 50 50
43 npoly 0 0
44 nmono 20 20
45 internal_poly 0 0
46 nfil 0

In [90]:
int.from_bytes(vesicle.contents.tape.contents.curvature_model,sys.byteorder)

15

In [91]:
small_functions.get_tape_options(v.tape)

{'nshell': 10,
 'dmax': 1.7,
 'dmin_interspecies': 1.2,
 'stepsize': 0.15,
 'nxmax': 100,
 'nymax': 100,
 'nzmax': 100,
 'mcsweeps': 1000,
 'inititer': 0,
 'iterations': 10,
 'quiet': 'false',
 'xk0': 20,
 'xk2': -20,
 'number_of_vertices_with_c0': 50,
 'c0': 0.5,
 'd0': 0.5,
 'curvature_model': 15,
 'min_dihedral_angle_cosine': 0.1,
 'w': 1.0,
 'bond_model': 0,
 'F': 1.0,
 'force_model': 0,
 'vicsek_strength': 0.1,
 'vicsek_radius': 4.0,
 'adhesion_model': 0,
 'adhesion_geometry': 0,
 'adhesion_cutoff': 1,
 'adhesion_z': -5,
 'adhesion_radius': 5.0,
 'adhesion_scale': 5.0,
 'adhesion_factor': 2.0,
 'adhesion_strength': 1.0,
 'allow_center_mass_movement': 0,
 'force_balance_along_z_axis': 0,
 'pressure_switch': 0,
 'pressure': 0.0,
 'volume_switch': 0,
 'constvolprecision': 1e-14,
 'V0': 0,
 'xkV0': 1.0,
 'Vr': 1.0,
 'area_switch': 0,
 'A0': 0,
 'xkA0': 1.0,
 'R_nucleus': 0,
 'R_nucleusX': 0,
 'R_nucleusY': 0,
 'R_nucleusZ': 0,
 'plane_confinement_switch': 0,
 'plane_d': 10,
 'plane_F'